In [1]:
# Import Dependancies

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder

#Obtain all the data

store_data = pd.read_csv( '/kaggle/input/store-sales-time-series-forecasting/stores.csv')
train_data = pd.read_csv( '/kaggle/input/store-sales-time-series-forecasting/train.csv')
test_data = pd.read_csv( '/kaggle/input/store-sales-time-series-forecasting/test.csv')
oil_price = pd.read_csv( '/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays = pd.read_csv( '/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
submission = pd.read_csv( '/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

In [2]:
# Initialize the LabelEncoder

label_encoder = LabelEncoder()

#Preprocessing the data

train_data['Family_Encoded']=label_encoder.fit_transform(train_data['family'])
test_data['Family_Encoded']=label_encoder.fit_transform(test_data['family'])

#Drop categorical columns

test_data.drop(columns = ['family'], inplace = True)
train_data.drop(columns = ['family'], inplace = True)

In [3]:
train_data

,id,date,store_nbr,sales,onpromotion,Family_Encoded
0,0,2013-01-01,1,0.000,0,0
1,1,2013-01-01,1,0.000,0,1
2,2,2013-01-01,1,0.000,0,2
3,3,2013-01-01,1,0.000,0,3
4,4,2013-01-01,1,0.000,0,4
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,438.133,0,28
3000884,3000884,2017-08-15,9,154.553,1,29
3000885,3000885,2017-08-15,9,2419.729,148,30
3000886,3000886,2017-08-15,9,121.000,8,31


In [4]:
test_data

,id,date,store_nbr,onpromotion,Family_Encoded
0,3000888,2017-08-16,1,0,0
1,3000889,2017-08-16,1,0,1
2,3000890,2017-08-16,1,2,2
3,3000891,2017-08-16,1,20,3
4,3000892,2017-08-16,1,0,4
...,...,...,...,...,...
28507,3029395,2017-08-31,9,1,28
28508,3029396,2017-08-31,9,0,29
28509,3029397,2017-08-31,9,1,30
28510,3029398,2017-08-31,9,9,31


In [5]:
# Transform the Holidays Dataset to allow for easy use in ML models


#Convert Date to DateTime Format
holidays['date'] = pd.to_datetime(holidays['date'])

#Dropping the column as it has over 100 unique values.
holidays.drop(columns=['description'], inplace = True)

#Encoding the categorical columns

holidays['Holiday_type_encoded'] = label_encoder.fit_transform(holidays['type'])
holidays['Holiday_locale_encoded'] = label_encoder.fit_transform(holidays['locale'])
holidays['Transferred_Holiday_encoded'] = label_encoder.fit_transform(holidays['transferred'])
holidays['Region_encoded'] = label_encoder.fit_transform(holidays['locale_name'])

holidays.drop(columns=['type','locale','locale_name','transferred'], inplace= True)
holidays

,date,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded
0,2012-03-02,3,0,0,15
1,2012-04-01,3,2,0,2
2,2012-04-12,3,0,0,3
3,2012-04-14,3,0,0,12
4,2012-04-21,3,0,0,19
...,...,...,...,...,...
345,2017-12-22,0,1,0,4
346,2017-12-23,0,1,0,4
347,2017-12-24,0,1,0,4
348,2017-12-25,3,1,0,4


In [6]:
holidays.nunique()

date                           312
Holiday_type_encoded             6
Holiday_locale_encoded           3
Transferred_Holiday_encoded      2
Region_encoded                  24
dtype: int64

In [7]:
#Encoding the categorical columns

store_data['City_encoded']=label_encoder.fit_transform(store_data['city'])
store_data['State_encoded']=label_encoder.fit_transform(store_data['state'])
store_data['Store_type_encoded']=label_encoder.fit_transform(store_data['type'])

store_data.drop(columns = ['city','state','type'], inplace = True)

In [8]:
#Convert Date to DateTime Format in the oil_price dataset

oil_price['date'] = pd.to_datetime(oil_price['date'])

# Split the oil price data

oil_price_test = oil_price[oil_price['date']>=test_data['date'][0]]
oil_price_train = oil_price[oil_price['date']<train_data['date'].iloc[-1]]

In [9]:
oil_price_train

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1200,2017-08-08,49.07
1201,2017-08-09,49.59
1202,2017-08-10,48.54
1203,2017-08-11,48.81


In [10]:
#Convert Date to DateTime Format in the transactons dataset

transactions['date'] = pd.to_datetime(transactions['date'])
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [11]:
# Ensure 'date' columns are in datetime64[ns] format in all DataFrames
holidays['date'] = pd.to_datetime(holidays['date'])
train_data['date'] = pd.to_datetime(train_data['date'])
oil_price['date'] = pd.to_datetime(oil_price['date'])
transactions['date'] = pd.to_datetime(transactions['date'])

# Now perform the merges
train_final = pd.merge(holidays, train_data, on='date', how = 'right')
train_final = pd.merge(train_final, store_data, on='store_nbr', how='inner')
train_final = pd.merge(train_final, oil_price_train, on='date')
train_final = pd.merge(train_final, transactions, on=['date', 'store_nbr'], how='left')


In [12]:
# Convert date string feature to timestamps
train_final['date'] = pd.to_datetime(train_final['date'])
train_final['Day']=train_final['date'].dt.day
train_final['Month']=train_final['date'].dt.month
train_final['Year']=train_final['date'].dt.year
train_final['Day of the week']=train_final['date'].dt.dayofweek
train_final['Day of the year']=train_final['date'].dt.dayofyear

#Prepare the train data
train_final.drop(columns = ['date'], inplace = True)
train_final.fillna(0,inplace = True)

In [13]:
train_final.dtypes

Holiday_type_encoded           float64
Holiday_locale_encoded         float64
Transferred_Holiday_encoded    float64
Region_encoded                 float64
id                               int64
store_nbr                        int64
sales                          float64
onpromotion                      int64
Family_Encoded                   int64
cluster                          int64
City_encoded                     int64
State_encoded                    int64
Store_type_encoded               int64
dcoilwtico                     float64
transactions                   float64
Day                              int32
Month                            int32
Year                             int32
Day of the week                  int32
Day of the year                  int32
dtype: object

In [14]:
train_final

,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded,id,store_nbr,sales,onpromotion,Family_Encoded,cluster,City_encoded,State_encoded,Store_type_encoded,dcoilwtico,transactions,Day,Month,Year,Day of the week,Day of the year
0,3.0,1.0,0.0,4.0,0,1,0.000,0,0,13,18,12,3,0.00,0.0,1,1,2013,1,1
1,3.0,1.0,0.0,4.0,1,1,0.000,0,1,13,18,12,3,0.00,0.0,1,1,2013,1,1
2,3.0,1.0,0.0,4.0,2,1,0.000,0,2,13,18,12,3,0.00,0.0,1,1,2013,1,1
3,3.0,1.0,0.0,4.0,3,1,0.000,0,3,13,18,12,3,0.00,0.0,1,1,2013,1,1
4,3.0,1.0,0.0,4.0,4,1,0.000,0,4,13,18,12,3,0.00,0.0,1,1,2013,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174035,0.0,0.0,0.0,0.0,2999101,9,283.429,0,28,6,18,12,1,47.59,1971.0,14,8,2017,0,226
2174036,0.0,0.0,0.0,0.0,2999102,9,114.120,0,29,6,18,12,1,47.59,1971.0,14,8,2017,0,226
2174037,0.0,0.0,0.0,0.0,2999103,9,1348.425,7,30,6,18,12,1,47.59,1971.0,14,8,2017,0,226
2174038,0.0,0.0,0.0,0.0,2999104,9,182.000,11,31,6,18,12,1,47.59,1971.0,14,8,2017,0,226


In [15]:
# Ensure 'date' columns are in datetime64[ns] format in all DataFrames

test_data['date'] = pd.to_datetime(test_data['date'])

#Merge the Holidays, Store_data, Oil_prices, and test_data

test_final = pd.merge(holidays, test_data, on = 'date', how = 'outer')
test_final = pd.merge(test_final, store_data, on='store_nbr',how = 'inner')
test_final = pd.merge(test_final, oil_price_test, on = 'date', how = 'left')
test_final = pd.merge(test_final, transactions, on = ['date','store_nbr'], how = 'left')

In [16]:

# Convert date string feature to timestamps
test_final['date'] = pd.to_datetime(test_final['date'], format='mixed')
test_final['Day']=test_final['date'].dt.day
test_final['Month']= test_final['date'].dt.month
test_final['Year']=test_final['date'].dt.year
test_final['Day of the week']=test_final['date'].dt.dayofweek
test_final['Day of the year']=test_final['date'].dt.dayofyear

#Change Datatype of oil prices
test_final['dcoilwtico'] = test_final['dcoilwtico'].astype(float)

#Prepare the test data
test_final.drop(columns = ['date'], inplace = True)
test_final.fillna(0, inplace= True)
test_final['sales']=np.nan

test_final


,Holiday_type_encoded,Holiday_locale_encoded,Transferred_Holiday_encoded,Region_encoded,id,store_nbr,onpromotion,Family_Encoded,cluster,City_encoded,State_encoded,Store_type_encoded,dcoilwtico,transactions,Day,Month,Year,Day of the week,Day of the year,sales
0,0.0,0.0,0.0,0.0,3000888.0,1.0,0.0,0.0,13,18,12,3,46.80,0.0,16,8,2017,2,228,NaN
1,0.0,0.0,0.0,0.0,3000889.0,1.0,0.0,1.0,13,18,12,3,46.80,0.0,16,8,2017,2,228,NaN
2,0.0,0.0,0.0,0.0,3000890.0,1.0,2.0,2.0,13,18,12,3,46.80,0.0,16,8,2017,2,228,NaN
3,0.0,0.0,0.0,0.0,3000891.0,1.0,20.0,3.0,13,18,12,3,46.80,0.0,16,8,2017,2,228,NaN
4,0.0,0.0,0.0,0.0,3000892.0,1.0,0.0,4.0,13,18,12,3,46.80,0.0,16,8,2017,2,228,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,0.0,0.0,0.0,0.0,3029395.0,9.0,1.0,28.0,6,18,12,1,47.26,0.0,31,8,2017,3,243,NaN
28508,0.0,0.0,0.0,0.0,3029396.0,9.0,0.0,29.0,6,18,12,1,47.26,0.0,31,8,2017,3,243,NaN
28509,0.0,0.0,0.0,0.0,3029397.0,9.0,1.0,30.0,6,18,12,1,47.26,0.0,31,8,2017,3,243,NaN
28510,0.0,0.0,0.0,0.0,3029398.0,9.0,9.0,31.0,6,18,12,1,47.26,0.0,31,8,2017,3,243,NaN


In [17]:
train_final.nunique()

Holiday_type_encoded                 5
Holiday_locale_encoded               3
Transferred_Holiday_encoded          2
Region_encoded                      24
id                             2141964
store_nbr                           54
sales                           284471
onpromotion                        330
Family_Encoded                      33
cluster                             17
City_encoded                        22
State_encoded                       16
Store_type_encoded                   5
dcoilwtico                         994
transactions                      4274
Day                                 31
Month                               12
Year                                 5
Day of the week                      5
Day of the year                    364
dtype: int64

In [18]:
test_final.columns

Index(['Holiday_type_encoded', 'Holiday_locale_encoded',
       'Transferred_Holiday_encoded', 'Region_encoded', 'id', 'store_nbr',
       'onpromotion', 'Family_Encoded', 'cluster', 'City_encoded',
       'State_encoded', 'Store_type_encoded', 'dcoilwtico', 'transactions',
       'Day', 'Month', 'Year', 'Day of the week', 'Day of the year', 'sales'],
      dtype='object')

In [19]:
train_final.columns

Index(['Holiday_type_encoded', 'Holiday_locale_encoded',
       'Transferred_Holiday_encoded', 'Region_encoded', 'id', 'store_nbr',
       'sales', 'onpromotion', 'Family_Encoded', 'cluster', 'City_encoded',
       'State_encoded', 'Store_type_encoded', 'dcoilwtico', 'transactions',
       'Day', 'Month', 'Year', 'Day of the week', 'Day of the year'],
      dtype='object')

In [20]:
# Set the test and train data

X_train, X_test, y_train, y_test = train_test_split(train_final.drop(columns=['sales']), train_final['sales'], test_size=0.25, random_state=4)

In [21]:
#Using lightgbm 
import lightgbm as lgb

# Define parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 62,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
}


# Convert training data to a LightGBM Dataset
train_data1 = lgb.Dataset(X_train, label=y_train)

# Train the LightGBM model
model = lgb.train(params, train_data1, num_boost_round=175)

# Prediction
y_pred1 = model.predict(X_test)

# Post-process the predictions to replace negative values with zero
y_pred_non_negative = np.maximum(y_pred1, 0)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.421010 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1462
[LightGBM] [Info] Number of data points in the train set: 1630530, number of used features: 19
[LightGBM] [Info] Start training from score 322.539138


In [22]:
# Calculate RMSLE

rmsle = mean_squared_log_error(y_true=y_test,y_pred = y_pred_non_negative)
    
print("Root Mean Squared Logarithmic Error:", rmsle)
print("Light GBM predicted " + str(len(y_pred1[y_pred1<0])/len(y_pred1)* 100) + "% negative resutls")

Root Mean Squared Logarithmic Error: 2.57591924675898
Light GBM predicted 15.797133447406672% negative resutls


In [23]:
#Using XGBoost
import xgboost as xgb


# Define parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 20,
    'learning_rate': 0.1,
    'subsample': 1
}

# Define XGBoost DMatrix
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test, label=y_test)

# Train the XGBoost model
model2 = xgb.train(params, train_dmatrix, num_boost_round=200, evals=[(test_dmatrix, 'test')], early_stopping_rounds=5)

# Prediction
y_pred2 = model2.predict(test_dmatrix)

# Post-process the predictions to replace negative values with zero
y_pred_non_negative = np.maximum(y_pred2, 0)

[0]	test-rmse:874.95877
[1]	test-rmse:795.36429
[2]	test-rmse:724.11000
[3]	test-rmse:660.53537
[4]	test-rmse:604.16400
[5]	test-rmse:554.05077
[6]	test-rmse:509.67097
[7]	test-rmse:470.36945
[8]	test-rmse:435.68029
[9]	test-rmse:405.46122
[10]	test-rmse:379.08563
[11]	test-rmse:355.96445
[12]	test-rmse:336.21921
[13]	test-rmse:319.01176
[14]	test-rmse:304.35517
[15]	test-rmse:292.09071
[16]	test-rmse:281.01487
[17]	test-rmse:271.67828
[18]	test-rmse:264.11764
[19]	test-rmse:257.71224
[20]	test-rmse:252.45894
[21]	test-rmse:248.07680
[22]	test-rmse:244.24698
[23]	test-rmse:241.39551
[24]	test-rmse:238.90330
[25]	test-rmse:236.91814
[26]	test-rmse:235.28303
[27]	test-rmse:233.76421
[28]	test-rmse:232.55433
[29]	test-rmse:231.80806
[30]	test-rmse:231.06471
[31]	test-rmse:230.77869
[32]	test-rmse:230.58644
[33]	test-rmse:230.56881
[34]	test-rmse:230.63890
[35]	test-rmse:230.78688
[36]	test-rmse:230.94758
[37]	test-rmse:230.86016


In [24]:
# Calculate RMSLE for XGBoost

rmsle = mean_squared_log_error(y_true=y_test,y_pred = y_pred_non_negative)
    
print("Root Mean Squared Logarithmic Error:", rmsle)
print("XGBoost predicted " + str((len(y_pred2[y_pred2<0])/len(y_pred2)* 100)) + "% negative resutls")

Root Mean Squared Logarithmic Error: 1.3776296162255537
XGBoost predicted 0.0029438280804401024% negative resutls


In [25]:
#Using Random Forrest Refressor
from sklearn.ensemble import RandomForestRegressor

# Model training with lower runtime
model3 = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=-1, random_state=2)
model3.fit(X_train, y_train)

# Model evaluation
y_pred3 = model3.predict(X_test)

# Post-process the predictions to replace negative values with zero
y_pred_non_negative = np.maximum(y_pred3, 0)



In [26]:
# Calculate RMSLE

rmsle = mean_squared_log_error(y_true=y_test,y_pred = y_pred_non_negative)

print("Root Mean Squared Logarithmic Error:", rmsle)
print("Random Forrest Regressors predicted " + str(len(y_pred3[y_pred3<0])/len(y_pred3)* 100) + "% negative resutls")

Root Mean Squared Logarithmic Error: 0.1676760281089549
Random Forrest Regressors predicted 0.0% negative resutls


In [27]:
# Predict the submission set with random forrest regressors

sample1 = model3.predict(test_final.drop(columns = ['sales']))

In [28]:
# Prepare the submission file and present sales as a whole number 
submission['sales'] = sample1.round(0)
submission

,id,sales
0,3000888,4.0
1,3000889,0.0
2,3000890,5.0
3,3000891,2803.0
4,3000892,7.0
...,...,...
28507,3029395,1050.0
28508,3029396,5.0
28509,3029397,104.0
28510,3029398,284.0
